# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 11:08AM,237641,22,615207,"NBTY Global, Inc.",Released
1,May 19 2022 11:06AM,237640,22,615208,"NBTY Global, Inc.",Released
2,May 19 2022 11:04AM,237639,22,615209,"NBTY Global, Inc.",Released
3,May 19 2022 11:03AM,237638,12,U0922,"Uniderm USA, Inc",Released
4,May 19 2022 10:41AM,237637,19,GP52022,"Granules Pharmaceuticals, Inc.",Executing
5,May 19 2022 10:32AM,237636,10,0085682677,ISDIN Corporation,Released
6,May 19 2022 10:32AM,237636,10,0085682714,ISDIN Corporation,Released
7,May 19 2022 10:32AM,237636,10,0085682713,ISDIN Corporation,Released
8,May 19 2022 10:32AM,237636,10,0085682716,ISDIN Corporation,Released
9,May 19 2022 10:32AM,237636,10,0085682715,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,237637,Executing,1
51,237638,Released,1
52,237639,Released,1
53,237640,Released,1
54,237641,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237637,NaN,1.0,NaN
237638,NaN,NaN,1.0
237639,NaN,NaN,1.0
237640,NaN,NaN,1.0
237641,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237550,0.0,0.0,1.0
237557,0.0,13.0,15.0
237566,0.0,0.0,4.0
237567,0.0,0.0,1.0
237571,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237550,0,0,1
237557,0,13,15
237566,0,0,4
237567,0,0,1
237571,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237550,0,0,1
1,237557,0,13,15
2,237566,0,0,4
3,237567,0,0,1
4,237571,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237550,,,1
1,237557,,13,15
2,237566,,,4
3,237567,,,1
4,237571,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc."
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc."
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc."
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc"
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc."
5,May 19 2022 10:32AM,237636,10,ISDIN Corporation
18,May 19 2022 10:17AM,237635,18,Innova Softgel LLC
19,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.
34,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.
59,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",,,1
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",,,1
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",,,1
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc",,,1
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc.",,1,
5,May 19 2022 10:32AM,237636,10,ISDIN Corporation,,,13
6,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,,,1
7,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,,,15
8,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,,4,21
9,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,,4,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",1,,
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",1,,
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",1,,
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc",1,,
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc.",,1,
5,May 19 2022 10:32AM,237636,10,ISDIN Corporation,13,,
6,May 19 2022 10:17AM,237635,18,Innova Softgel LLC,1,,
7,May 19 2022 10:11AM,237633,10,Eywa Pharma Inc.,15,,
8,May 19 2022 10:11AM,237632,10,Eywa Pharma Inc.,21,4,
9,May 19 2022 10:11AM,237634,20,Alumier Labs Inc.,9,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",1,,
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",1,,
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",1,,
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc",1,,
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc",1.0,NaN,NaN
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 11:08AM,237641,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,May 19 2022 11:06AM,237640,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,May 19 2022 11:04AM,237639,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,May 19 2022 11:03AM,237638,12,"Uniderm USA, Inc",1.0,0.0,0.0
4,May 19 2022 10:41AM,237637,19,"Granules Pharmaceuticals, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3326608,220.0,37.0,5.0
12,237638,1.0,0.0,0.0
15,950487,23.0,5.0,1.0
16,712775,3.0,0.0,0.0
18,1900774,18.0,0.0,0.0
19,950495,10.0,28.0,8.0
20,1187910,72.0,17.0,0.0
21,237575,0.0,1.0,0.0
22,950491,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3326608,220.0,37.0,5.0
1,12,237638,1.0,0.0,0.0
2,15,950487,23.0,5.0,1.0
3,16,712775,3.0,0.0,0.0
4,18,1900774,18.0,0.0,0.0
5,19,950495,10.0,28.0,8.0
6,20,1187910,72.0,17.0,0.0
7,21,237575,0.0,1.0,0.0
8,22,950491,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,220.0,37.0,5.0
1,12,1.0,0.0,0.0
2,15,23.0,5.0,1.0
3,16,3.0,0.0,0.0
4,18,18.0,0.0,0.0
5,19,10.0,28.0,8.0
6,20,72.0,17.0,0.0
7,21,0.0,1.0,0.0
8,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,220.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,3.0
4,18,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,5.0,0.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0
Executing,37.0,0.0,5.0,0.0,0.0,28.0,17.0,1.0,1.0
Released,220.0,1.0,23.0,3.0,18.0,10.0,72.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,5.0,0.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0
1,Executing,37.0,0.0,5.0,0.0,0.0,28.0,17.0,1.0,1.0
2,Released,220.0,1.0,23.0,3.0,18.0,10.0,72.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,5.0,0.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0
1,Executing,37.0,0.0,5.0,0.0,0.0,28.0,17.0,1.0,1.0
2,Released,220.0,1.0,23.0,3.0,18.0,10.0,72.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()